## Capturing Images for the dataset

For numbered cards 

In [ ]:
import cv2
vc = cv2.VideoCapture(1)
img_count = 0
special_cards = ['skip', 'reverse','picktwo']
while vc.isOpened():
    rval, frame = vc.read()    # read video frames again at each loop, as long as the stream is open
    cv2.imshow("stream", frame)# display each frame as an image, "stream" is the name of the window
    key = cv2.waitKey(1)       # allows user intervention without stopping the stream (pause in ms)
    if key == 32:
        img_name = "./unocards/g{}.jpg".format(img_count)
        cv2.imwrite(img_name, frame)
        img_count += 1
    elif key == 27:              # exit on ESC
        break
cv2.destroyWindow("stream")    # close image window upon exit
vc.release()   

For Special Cards

In [ ]:
import cv2
vc = cv2.VideoCapture(1)
img_count = 0
i = 0
special_cards = ['skip', 'reverse','picktwo']
wild_cards = ['pickfour', 'colorswitch']

while vc.isOpened():
    rval, frame = vc.read()    # read video frames again at each loop, as long as the stream is open
    cv2.imshow("stream", frame)# display each frame as an image, "stream" is the name of the window
    key = cv2.waitKey(1)       # allows user intervention without stopping the stream (pause in ms)
    if key == 32:
            img_name = "./unocards/{}.jpg".format(wild_cards[i])
            i+=1
            cv2.imwrite(img_name, frame)
            
    elif key == 27:              # exit on ESC
        break
cv2.destroyWindow("stream")    # close image window upon exit
vc.release()   

### Color Detection

In [7]:
import cv2
import numpy as np
img_colour = cv2.imread('./unocards/b1.jpg')   # open the saved image in colour

# Image Pre-processing
img = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)   # convert to B/W
img_HSV = cv2.cvtColor(img_colour, cv2.COLOR_BGR2HSV) # Convert to HSV
img_sm = cv2.blur(img, (7, 7))         # smoothing
thr_value, img_th = cv2.threshold(img_sm, 160, 255, cv2.THRESH_BINARY_INV)   # binarisation
# img_th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 35, 2)
kernel = np.ones((3, 3), np.uint8)
img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)      # morphology correction
img_canny = cv2.Canny(img_close, 100, 200)                          # edge detection
contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)   # extract contours on binarised image, not on canny

# Select the region of interest i.e the card and crop the background 
contour1 = contours[1]
x, y, w, h = cv2.boundingRect(contour1) # Find the bounding rectangle of contour 1

crop_img = img_colour[y:y+h, x:x+w] # Crop the image to the bounding rectangle

# Checking for color using HSV ranges

# Convert the image to HSV
hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)

# Define the range of colors for the Uno card

# Green color range
green_lower = np.array([45, 50, 50])
green_upper = np.array([75, 255, 255])
green_range = np.array([green_lower, green_upper])

# Yellow color range
yellow_lower = np.array([22, 50, 50])
yellow_upper = np.array([38, 255, 255])
yellow_range = np.array([yellow_lower, yellow_upper])

# Blue color range
blue_lower = np.array([100, 50, 50])
blue_upper = np.array([130, 255, 255])
blue_range = np.array([blue_lower, blue_upper])

# Red color range (two ranges to account for hue wraparound)
red_lower1 = np.array([0, 50, 50])
red_upper1 = np.array([10, 255, 255])
red_lower2 = np.array([170, 50, 50])
red_upper2 = np.array([180, 255, 255])
red_range = np.array([red_lower1, red_upper1, red_lower2, red_upper2])

# Create a mask for the red color
mask = cv2.inRange(hsv, red_lower1, red_upper1)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(crop_img,crop_img, mask= mask)

# Check if the card is red
if np.any(res):
    print("The Uno card is red")
else:
    print("The Uno card is not red")



The Uno card is not red
